In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder

import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import fasttext
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, Reshape
from tensorflow.keras.callbacks import EarlyStopping


2023-11-05 12:31:31.405708: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-05 12:31:31.412722: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 12:31:31.565275: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 12:31:31.567241: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-05 12:31:34.179075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
dataset = '../data/Hate Content Detection.csv'

In [3]:
df = pd.read_csv(dataset)
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
df.drop(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'], axis=1, inplace=True)
df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [5]:
df.drop_duplicates(inplace=True)

In [6]:
def cleanText(text):
    text = re.sub(r'RT @\w+: ', '', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text    

df['tweet'] = df['tweet'].apply(cleanText)
df.head()

,class,tweet
0,2,as a woman you shouldn t complain about cl...
1,1,boy dats cold tyga dwn bad for cuffin ...
2,1,rt urkindofbrand dawg you ever fu...
3,1,viva based she look like a tranny
4,1,the shit you hear about me might...


In [7]:
cv = CountVectorizer()
X = cv.fit_transform(df['tweet'])
y = df['class']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.toarray(), y_train, epochs=10, batch_size=32, validation_data=(X_test.toarray(), y_test))

model.save('../models/hateDetection.h5')
pickle.dump(cv, open('../models/cvhate.pkl', 'wb'))

2023-11-05 12:31:39.995232: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2023-11-05 12:31:39.995446: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: n00b
2023-11-05 12:31:39.995487: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: n00b
2023-11-05 12:31:39.996017: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.129.3
2023-11-05 12:31:39.996108: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-11-05 12:31:39.996129: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.113.1 does not match DSO version 535.129.3 -- cannot find working devices in this configuration


Epoch 1/10
620/620 [==============================] - 19s 29ms/step - loss: 0.3928 - accuracy: 0.8640 - val_loss: 0.2920 - val_accuracy: 0.9009
Epoch 2/10
620/620 [==============================] - 14s 23ms/step - loss: 0.1668 - accuracy: 0.9432 - val_loss: 0.3191 - val_accuracy: 0.8927
Epoch 3/10
620/620 [==============================] - 14s 22ms/step - loss: 0.0808 - accuracy: 0.9727 - val_loss: 0.4003 - val_accuracy: 0.8878
Epoch 4/10
620/620 [==============================] - 14s 23ms/step - loss: 0.0413 - accuracy: 0.9865 - val_loss: 0.4819 - val_accuracy: 0.8777
Epoch 5/10
620/620 [==============================] - 15s 24ms/step - loss: 0.0241 - accuracy: 0.9927 - val_loss: 0.5395 - val_accuracy: 0.8842
Epoch 6/10
620/620 [==============================] - 15s 24ms/step - loss: 0.0167 - accuracy: 0.9952 - val_loss: 0.6118 - val_accuracy: 0.8737
Epoch 7/10
620/620 [==============================] - 15s 24ms/step - loss: 0.0117 - accuracy: 0.9961 - val_loss: 0.6596 - val_accuracy:

: 